MECA.MECA_MOVIMIENTOS
MECA.CAPF_CLIENTES
MECA.MECA_CARTERAS
MECA.MECA_CONCEPTOS
MECA.MECA_OPERACIONES
MECA.MECA_SERIES
MECA.MECA_SECTORES
MECA.MECA_EMISORAS

In [1]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

# Get the data and create the output directory (Extraction)

In [6]:
path_dir_data = "/home/ale1726/proyects/datalake/clientes/data/productos/MECA/data"

In [ ]:
# Obtenemos la fecha actual para crear un directorio
# donde se guardaran los archivos de salida
date_now = datetime.now().strftime("%d_%m_%Y") 
path_dir_today =  os.path.join(path_dir_data, date_now)

# Creamos el directorio si no existe
os.makedirs(path_dir_today, exist_ok=True)

In [18]:
query_productos_meca = """ 
WITH CLIENTES_VIG AS (
    SELECT CFCL.CLI_CLAVE_FFON, CFCL.CLI_CLAVE, CFCL.CLI_NOMBRE, CTDC.TDC_DESCRIPCION AS TIPO_CLIENTE
	FROM MECA.CAPF_CLIENTES CFCL
	LEFT JOIN MECA.CAPF_TIPOS_DE_CLIENTES CTDC ON CFCL.CLI_TDC_CLAVE = CTDC.TDC_CLAVE
	WHERE CFCL.CLI_EST_CLAVE = 'VIG' AND CFCL.CLI_NOMBRE != '0'
), OPE_C1 AS (
	SELECT MO.*, MI.INS_DESCRIPCION INSTRUMENTO
	FROM MECA.MECA_OPERACIONES MO
	LEFT JOIN MECA.MECA_INSTRUMENTOS MI ON MO.OPE_INS_CLAVE = MI.INS_CLAVE
), OPE_C2 AS (
	SELECT OP.*, CAT.TMO_DESCRIPCION TIPO_MOVIMIENTO
	FROM OPE_C1 OP
	LEFT JOIN MECA.MECA_TIPOS_MOVIMIENTO CAT ON OP.OPE_TMO_CLAVE = CAT.TMO_CLAVE
), TAB_OPE AS(
	SELECT OPE_FOLIO, OPE_TYPE, OPE_INS_CLAVE ,INSTRUMENTO, OPE_SER_NUM_SERIE, OPE_SER_EMI_CLAVE, 
		OPE_TMO_CLAVE, TIPO_MOVIMIENTO, OPE_CLI_CLAVE, OPE_DIV_CLAVE_SWIFT, OPE_NUM_TITULOS, OPE_PRECIO_POSTURA
		OPE_IMPORTE_TITULOS, OPE_FECHA_PACTADA, OPE_FECHA_SALDADA, OPE_FECHA_ALTA, OPE_FECHA_VENCIMIENTO, OPE_TIPO_PRESTAMO,
		OPE_TIPO_CAMBIO, OPE_TASA_PRESTAMO, OPE_DESCUENTO_PREST, OPE_PLAZO, OPE_PORC_COMISION, OPE_PORC_IVA, OPE_IMPORTE_COMISION,
		OPE_IMPORTE_TOTAL, OPE_IMPORTE_PRIMA, OPE_PRECIO_PRESTAMO, OPE_PRECIO_PACTADA
	FROM OPE_C2
), TAB_FINAL AS (
	SELECT CV.*, TAOP.*
	FROM CLIENTES_VIG CV
	LEFT JOIN TAB_OPE TAOP ON TAOP.OPE_CLI_CLAVE = CV.CLI_CLAVE
) SELECT * FROM TAB_FINAL
"""

In [19]:
path_dir_productos = get_table(path_dir_today, db_meca, "productos_clientes_meca", query = query_productos_meca)

# Transform the data and output the transformation

In [55]:
path_dir_output_transformation = os.path.join("/home/ale1726/proyects/datalake/clientes/data/productos/MECA/agrupados", date_now)
os.makedirs(path_dir_output_transformation, exist_ok=True)
path_dir_output_transformation

'/home/ale1726/proyects/datalake/clientes/data/productos/MECA/agrupados/21_04_2025'

In [ ]:
""" 
agrupaddo_anual_mensual = (
    informacion_garantia
    .groupby(['AÑO', 'MES'], as_index=False)
    .agg(
        INTERMEDIARIOS=('CLIENTE', 'nunique'),  # Número de clientes distintos
        GARANTIAS=('NUMERO_DE_GARANTIA', 'count'),  # Número total de garantías
        SUMA_MENSUAL_MONTO_GARANTIZADO = ('MONTO_GARANTIZADO', 'sum'),
        TOTAL_MENSUAL_GANANCIA = ('GANANCIA','sum'),
        PROMEDIO_MENSUAL_COMISION = ('PORCENTAJE_COMISION',lambda x: round(x.mean() , 4))
    )
    .assign(PROMEDIO_GARANTIAS_X_INTERMEDIARIO = lambda df: round(df['GARANTIAS'] / df['INTERMEDIARIOS']),
            PROMEDIO_MENSUAL_MONTO_GARANTIZADO_X_GARANTIA = lambda df: round(df['SUMA_MENSUAL_MONTO_GARANTIZADO']/ df['GARANTIAS']))  # Calculo del promedio
    .sort_values(by=['AÑO', 'MES'])
)

"""

In [38]:
productos_meca =  pd.read_csv(path_dir_productos, low_memory=False)

In [39]:
productos_meca["OPE_FECHA_ALTA"] = pd.to_datetime(productos_meca["OPE_FECHA_ALTA"])
productos_meca["OPE_FECHA_PACTADA"] = pd.to_datetime(productos_meca["OPE_FECHA_PACTADA"])
productos_meca["OPE_FECHA_SALDADA"] = pd.to_datetime(productos_meca["OPE_FECHA_SALDADA"])
productos_meca["OPE_FECHA_VENCIMIENTO"] = pd.to_datetime(productos_meca["OPE_FECHA_VENCIMIENTO"])

In [64]:
productos_meca["AÑO_FALTA"] = productos_meca["OPE_FECHA_ALTA"].dt.year
productos_meca["AÑO_FPACTADA"] = productos_meca["OPE_FECHA_PACTADA"].dt.year
productos_meca["AÑO_FSALDADA"] = productos_meca["OPE_FECHA_SALDADA"].dt.year
productos_meca["AÑO_FVENCIMIENTO"] = productos_meca["OPE_FECHA_VENCIMIENTO"].dt.year
productos_meca["MES_FALTA"] = productos_meca["OPE_FECHA_ALTA"].dt.month
productos_meca["MES_FALTA_STR"] = productos_meca["OPE_FECHA_ALTA"].dt.month_name()        


productos_meca["MES_FPACTADA"] = productos_meca["OPE_FECHA_PACTADA"].dt.month
productos_meca["MES_FSALDADA"] = productos_meca["OPE_FECHA_SALDADA"].dt.month
productos_meca["MES_FVENCIMIENTO"] = productos_meca["OPE_FECHA_VENCIMIENTO"].dt.month

#productos_meca["MES"] = productos_meca["OPE_FECHA_ALTA"].dt.month
#productos_meca["MES"] = productos_meca["OPE_FECHA_ALTA"].dt.month_name()        

In [35]:
productos_meca.columns

Index(['CLI_CLAVE_FFON', 'CLI_CLAVE', 'CLI_NOMBRE', 'TIPO_CLIENTE',
       'OPE_FOLIO', 'OPE_TYPE', 'OPE_INS_CLAVE', 'INSTRUMENTO',
       'OPE_SER_NUM_SERIE', 'OPE_SER_EMI_CLAVE', 'OPE_TMO_CLAVE',
       'TIPO_MOVIMIENTO', 'OPE_CLI_CLAVE', 'OPE_DIV_CLAVE_SWIFT',
       'OPE_NUM_TITULOS', 'OPE_IMPORTE_TITULOS', 'OPE_FECHA_PACTADA',
       'OPE_FECHA_SALDADA', 'OPE_FECHA_ALTA', 'OPE_FECHA_VENCIMIENTO',
       'OPE_TIPO_PRESTAMO', 'OPE_TIPO_CAMBIO', 'OPE_TASA_PRESTAMO',
       'OPE_DESCUENTO_PREST', 'OPE_PLAZO', 'OPE_PORC_COMISION', 'OPE_PORC_IVA',
       'OPE_IMPORTE_COMISION', 'OPE_IMPORTE_TOTAL', 'OPE_IMPORTE_PRIMA',
       'OPE_PRECIO_PRESTAMO', 'OPE_PRECIO_PACTADA', 'AÑO_FALTA',
       'AÑO_FPACTADA', 'AÑO_FSALDADA', 'AÑO_FVENCIMIENTO', 'MES_FALTA',
       'MES_FPACTADA', 'MES_FSALDADA', 'MES_FVENCIMIENTO'],
      dtype='object')

In [71]:
productos_meca.value_counts('TIPO_CLIENTE')

TIPO_CLIENTE
CASA DE BOLSA                156118
BANCO                           426
CLIENTES CORPORATIVOS            41
CASA DE CAMBIO                   14
AREAS NAFIN                       9
PERSONA FISICA                    4
SUCURSAL                          4
PERSONA MORAL                     2
PERSONA MORAL BANCOMER 22         1
Name: count, dtype: int64

In [52]:
agrupado_operaciones_anuales = (
    productos_meca.groupby(['AÑO_FALTA'], as_index=False)
    .agg(
        num_operaciones=('OPE_FOLIO', 'count'), 
        monto_anuales_sum=('OPE_IMPORTE_TOTAL', 'sum'), 
        monto_anuales_min=('OPE_IMPORTE_TOTAL', 'min'), 
        monto_anuales_max=('OPE_IMPORTE_TOTAL', 'max'), 
        monto_anuales_mean=('OPE_IMPORTE_TOTAL', 'mean'), 
        monto_anuales_median=('OPE_IMPORTE_TOTAL', 'median')
    ).sort_values(by=['AÑO_FALTA'], ascending=True)
)

In [54]:
agrupado_operaciones_anuales.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales.csv"), index=False)

In [65]:
agrupado_operaciones_anuales_mes = (
    productos_meca.groupby(['AÑO_FALTA','MES_FALTA','MES_FALTA_STR'], as_index=False)
    .agg(
        num_operaciones=('OPE_FOLIO', 'count'), 
        monto_anuales_sum=('OPE_IMPORTE_TOTAL', 'sum'), 
        monto_anuales_min=('OPE_IMPORTE_TOTAL', 'min'), 
        monto_anuales_max=('OPE_IMPORTE_TOTAL', 'max'), 
        monto_anuales_mean=('OPE_IMPORTE_TOTAL', 'mean'), 
        monto_anuales_median=('OPE_IMPORTE_TOTAL', 'median')
    ).sort_values(by=['AÑO_FALTA','MES_FALTA'], ascending=True)
)

In [68]:
agrupado_operaciones_anuales_mes.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales_mes.csv"), index=False)